In [28]:
# This code loops through all AEO Price files in a single folder location and creates one harmonized dataframe with all prices
# Reads excel input. Splits data into three different dataframes, manipulates and creates unique fields and concats all
#  dataframes back together

# AEO Price File link: https://www.eia.gov/outlooks/aeo/tables_ref.php
#  -- see "Energy Prices by Sector", tables 3.1 to 3.9

# for reading the excel files
import glob
import pandas as pd
import numpy as np

from pandas import Series, DataFrame

# Initialize dataframe
DF_AEOprices = []

# Loop that goes through price files and pulls all relevant prices
for file in glob.iglob('AEO21*.xlsx'):
    
    # read excel file. 
    df = pd.read_excel(file,skiplines=16, na_values = '- -').dropna().reset_index().iloc[1:,:-1]
    
    # creates column names for reference
    column_index = [1,2]
    new_column_names = ['UniqueID', 'Fuel']
    old_column_names = df.columns[column_index]
    df.rename(columns=dict(zip(old_column_names, new_column_names)),inplace=True)

    # Splits string in first column into three columns for unique reference
    u = df['UniqueID'].str.split(":|_",n=3,expand = True)
    u1 = u.loc[u[1]!='nom']
    u2 = u.loc[u[1]=='nom']
    u1.insert(1,"Currency Temp","real")
    u1 = u1.iloc[:,:-2]
    u2 = u2.iloc[:,:-1]
    
    # Rename particular columns
    column_index_u1 = [0,1,2]
    new_column_names_u1 = ['Region', 'Currency', 'Sector']
    old_column_names_u1 = u1.columns[column_index_u1]
    u1.rename(columns=dict(zip(old_column_names_u1, new_column_names_u1)),inplace=True)
    
    # Rename particular columns
    column_index_u2 = [0,1,2]
    new_column_names_u2 = ['Region', 'Currency', 'Sector']
    old_column_names_u2 = u2.columns[column_index_u2]
    u2.rename(columns=dict(zip(old_column_names_u2, new_column_names_u2)),inplace=True)
    
    DF_U_Final = u1.append(u2)
    
    # changes data series into float64
    s=df[df.columns[3:]]
    s.astype(np.float64)
    
    # takes "fuel" column from original dataframe
    t=df.iloc[:,2]
    
    # Concats all three tables to one
    DF_CONCAT = pd.concat([DF_U_Final, t, s],axis=1)
    
    # Appends latest dataframe to original
    DF_AEOprices.append(DF_CONCAT)

# Concats latest dataframe to original, initialized dataframe
DF_AEOprices = pd.concat(DF_AEOprices).reset_index()
    


In [30]:
DF_AEOprices.tail(100)

,index,Region,Currency,Sector,Fuel,2020,2021,2022,2023,2024,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
695,73,PRC004,nom,Avg,E85 3/,20.0884,21.3445,23.4426,23.721992,23.8873,...,44.119968,45.415470,46.729950,48.046146,48.892349,50.914719,52.528809,54.218056,55.638611,57.325687
696,74,PRC004,nom,Avg,Motor Gasoline 4/,18.0049,18.8371,19.2075,19.767269,19.903,...,36.758858,37.838215,38.948170,40.048218,40.738098,42.426392,43.778049,45.179008,46.355705,47.770390
697,75,PRC004,nom,Avg,Jet Fuel 5/,9.55741,10.4413,11.5545,13.004985,14.2607,...,29.509026,30.440018,31.775455,32.678501,33.550056,35.010326,36.173080,37.145184,38.432804,39.624954
698,76,PRC004,nom,Avg,Distillate Fuel Oil,17.486,17.3688,17.8995,19.267567,19.8236,...,34.189404,35.153744,36.353806,37.202232,38.138496,39.641850,40.914227,41.991226,43.399391,44.601849
699,77,PRC004,nom,Avg,Residual Fuel Oil,4.83763,3.93557,5.15282,6.905498,8.26015,...,21.089388,21.770821,22.325922,23.047457,23.626574,24.544281,25.428766,26.277805,27.218313,28.206074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,81,PRC007,nom,Industrial,Industrial 1/,52.3515,59.0139,67.0084,71.272636,75.5406,...,176.287186,182.985016,190.492065,197.119614,202.992172,212.413788,221.828384,230.880585,241.396515,252.182785
791,82,PRC007,nom,TransNonRenw,Transportation,64.0114,69.917,73.3971,78.823959,81.937,...,158.587814,164.482941,171.522263,177.633545,182.873810,191.929688,199.736252,207.231308,214.988205,223.006088
792,83,PRC007,nom,TotalNon-Ren,Total Non-Renewable Expenditures,163.577,178.356,192.135,202.615952,210.949,...,428.564667,443.955170,461.244873,477.073883,491.587891,513.600952,534.661804,555.229614,577.205627,600.189331
793,84,PRC007,nom,TransRenewEx,Transportation Renewable Expenditures,0.09308,0.102586,0.101908,0.107478,0.106915,...,0.176633,0.185018,0.194190,0.203833,0.212469,0.225575,0.238351,0.251856,0.265316,0.280956
